In [73]:
import requests
from collections import namedtuple
# import validators
from bs4 import BeautifulSoup, NavigableString
from halo import Halo
def get_soup(url):
    '''
    Returns a BeautifulSoup version of a web page.
    '''
    response = requests.get(url)
    content = response.text
    return BeautifulSoup(content)

def get_links(table):
    '''
    Retrieves links from an embedded tabular structure.
    '''
    links = set()
    for child in table.children:
        for link in child.find_all('a', href=True):
            links.add(link['href'])

    return links

In [50]:
url = 'http://christmas-specials.wikia.com/'
    #data='xmas'
    # if not validators.url(url):
    #     print('Invalid URL')
    # path = pathlib.Path(data)

spinner = Halo(text='Fetching article list . . .')
spinner.start()
    
soup = get_soup(url + '/wiki/Special:AllPages')
spinner.succeed(text='Fetching list ===> Done')

'''Process the articles'''

articles = set()

spinner = Halo(text='Processing articles . . .')
spinner.start()

table_chunks = soup.find('table', 
        {'class': ['allpageslist', 'mw-allpages-table-chunk']})
    
chunks = set(get_links(table_chunks))

chunks_done = 0
chunks_left = len(chunks)

⠋ Fetching article list . . .⠙ Fetching article list . . .⠹ Fetching article list . . .⠸ Fetching article list . . .⠼ Fetching article list . . .⠴ Fetching article list . . .✔ Fetching list ===> Done
⠙ Processing articles . . .

In [105]:
# proof of concept

article = '/wiki/Mr._Monk_and_the_Man_Who_Shot_Santa_Claus'

def get_article_data(article, url='http://christmas-specials.wikia.com/'):
    Article = namedtuple('Article', 'title contents categories')
    article_soup = get_soup(url + article)
    article_title = article_soup.find('h1').contents
    article_contents = article_soup.find_all('div', {'class': ['mw-content-ltr', 'mw-content-text', 'mw-collapsible', 'mw-made-collapsible']})
    article_categories = [li['data-name'] for li in article_soup.find_all('li', {'class': 'category normal', 'data-type': 'normal'})]
    article = Article(title=article_title, contents=article_contents, categories=article_categories)
    return article

monk = get_article_data(article)

In [114]:
monk.contents

/tr></table>
 <p>"<b>Mr. Monk and the Man Who Shot Santa Claus</b>" is the third <a href="/wiki/Christmas" title="Christmas">Christmas</a> episode of the USA Network original series <i><a class="extiw" href="http://en.wikipedia.org/wiki/Monk_(TV_series)" title="wikipedia:Monk (TV series)">Monk</a></i>, produced and aired as the tenth episode of the its sixth season.
 </p>
 <h2><span class="mw-headline" id="Synopsis">Synopsis</span></h2>
 <table border="0" cellpadding="3" cellspacing="3" style="margin: 0 auto; border: 1px solid darkred; background-color: #dc4f4f; color:white;">
 <tr>
 <td> <center><i><b>SPOILER:</b></i> <i>Plot details or story follow.</i></center>
 </td></tr></table>
 <p>Another Christmas shopping season, which fills Adrian Monk's heart with dread. <a class="extiw" href="http://en.wikipedia.org/wiki/Natalie_Teeger" title="wikipedia:Natalie Teeger">Natalie Teeger</a> is fighting a Christmastime traffic jam in San Francisco to get Monk to his appointment with <a class="e

In [130]:
# NOT THE RIGHT WAY TO DO THIS, i am screwing around
for item in monk.contents:
     print(item.find_all('p')) # get content paragraphs
    # print(item.find_all('table'))
     print([i for i in item.find('h2').next_siblings]) # get related articles

[<p>He's a... bad Santa.<br/>
Very bad, bad, Santa.<br/>
Bad, bad, bad Santa.
</p>, <p style="font-size:smaller;line-height:1em;text-align: right"><cite style="font-style:normal;">— Adrian Monk</cite></p>, <p>"<b>Mr. Monk and the Man Who Shot Santa Claus</b>" is the third <a href="/wiki/Christmas" title="Christmas">Christmas</a> episode of the USA Network original series <i><a class="extiw" href="http://en.wikipedia.org/wiki/Monk_(TV_series)" title="wikipedia:Monk (TV series)">Monk</a></i>, produced and aired as the tenth episode of the its sixth season.
</p>, <p>Another Christmas shopping season, which fills Adrian Monk's heart with dread. <a class="extiw" href="http://en.wikipedia.org/wiki/Natalie_Teeger" title="wikipedia:Natalie Teeger">Natalie Teeger</a> is fighting a Christmastime traffic jam in San Francisco to get Monk to his appointment with <a class="extiw" href="http://en.wikipedia.org/wiki/Charles_Kroger" title="wikipedia:Charles Kroger">Dr. Kroger</a>, while her daughter <a

In [99]:
spinner.text = f'Processing chunks: {chunks_done}/{chunks_left}'

'''Process the chunks'''
while len(chunks) > 0:
    current_link = chunks.pop()
    soup = get_soup(url + current_link)
    current_table = soup.find('table', 
    {'class': ['allpageslist', 'mw-allpages-table-chunk']})
    if 'allpageslist' in current_table.get('class'):
        child_chunks = get_links(current_table)
        chunks = chunks.union(child_chunks)
        chunks_left += len(child_chunks)
    if 'mw-allpages-table-chunk' in current_table.get('class'):
        for child in current_table.children:
            if isinstance(child, NavigableString):
                continue
            else:
                for link in child.find_all('a', href=True):
                    article_link = link['href']
                    article = article_link.split('/')[-1]
                    articles.add((article, article.replace('_', ' ')))
      
    chunks_done += 1
    spinner.text = f'Processing chunks: {chunks_done}/{chunks_left}'

spinner.succeed(text='Processing chunks ===> Done')

✔ Processing chunks ===> Done


In [49]:
len(articles)

8684

In [100]:
spinner = Halo(text=f'Printing {len(articles)} articles')
spinner.start()

# for article in sorted(articles):
#    with open(f'{article}.txt', 'w') as outfile:
#        outfile.write(article)
#        outfile.write('\n')

spinner.succeed(text=f'Printing {len(articles)} articles. ===> Done')

⠋ Printing 8684 articles⠙ Printing 8684 articles✔ Printing 8684 articles. ===> Done
